In [ ]:
import pandas as pd
import numpy as np
import os
import time
import datetime
from openai import OpenAI
import seaborn as sns
import pymssql
import re
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
from threading import Thread
import functools
from tableone import TableOne
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu as mwu
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.statistics import multivariate_logrank_test
from sksurv.nonparametric import kaplan_meier_estimator
from sklearn.metrics import roc_curve, auc
import random
from threading import Thread
import functools
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, auc
from lifelines.utils import concordance_index

conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
openai.api_key = ""  # enter openai api key

# make synthetic data

In [ ]:
def find_lognormal_parameters(b, c, percentile_indecimal):
    # Calculate z-scores for 1% and 99%
    z_percentile = norm.ppf(percentile_indecimal)


    # The median of the lognormal is the exp of the mean (mu) of the underlying normal
    mu = np.log(b)

    # Use the equations derived from lognormal properties to solve for sigma
    # a = exp(mu + z_1 * sigma) and c = exp(mu + z_99 * sigma)
    sigma = (np.log(c) - mu) / z_percentile
    a = np.exp(mu + norm.ppf(0.001) * sigma)
    c = np.exp(mu + norm.ppf(0.999) * sigma)

    return mu, sigma, a, c

In [ ]:
age_median=50
age_99p=90
HDL_median=50
HDL_1p=20
LDL_median=120
LDL_99p=240
TG_median=120
TG_99p=500
HbA1c_median=5.2
HbA1c_99p=10
Cr_median=0.8
Cr_99p=2
Urate_median=5
Urate_99p=9
CRP_median=0.15
CRP_99p=2
SBP_median=140
SBP_99p=200
DBP_median=85
DBP_99p=120
BMI_median=25
BMI_99p=40
WHratio_median=0.85
WHratio_99p=1.1

In [ ]:
targetsize=20000
continuous_data_list=['age','HDL','LDL','TG','HbA1c','Cr','Urate','CRP','SBP','DBP','BMI','WHratio']
df_synthetic=pd.DataFrame()
for i in range(len(continuous_data_list)):
    totalsize=100000
    if continuous_data_list[i]!='HDL':
        percentilenum=0.99
        percentilestr='99'
    else:
        percentilenum=0.01
        percentilestr='1'
        
    b=globals()[   (continuous_data_list[i]+'_median'  )  ]
    c=globals()[   (continuous_data_list[i]+'_'+percentilestr+  'p'  )  ]
    x1,x2,p1,p99=find_lognormal_parameters(b, c,percentilenum)
    
    s = np.random.lognormal(x1, x2, totalsize)
    s2=[]
    for j in s:
        if j<=p99 and j>=p1:
            s2.append(j)
    if len(s2)<targetsize:
        print('===========error==============',continuous_data_list[i])
    s2=s2[:targetsize]
    df_synthetic[continuous_data_list[i]]=s2

sexlist=[]
CVDfirstrelativelist=[]
HTNlist=[]
DMlist=[]
Dyslipidlist=[]
Afiblist=[]
CKDlist=[]
for i in range(targetsize//2):
    sexlist.append(0)
    CVDfirstrelativelist.append(0)
    HTNlist.append(0)
    DMlist.append(0)
    Dyslipidlist.append(0)
    Afiblist.append(0)
    CKDlist.append(0)
for i in range(targetsize//2,targetsize):
    sexlist.append(1)
    CVDfirstrelativelist.append(1)
    HTNlist.append(1)
    DMlist.append(1)
    Dyslipidlist.append(1)
    Afiblist.append(1)
    CKDlist.append(1)
    
smokinglist=[]
for i in range(targetsize//3):
    smokinglist.append(1)
for i in range(targetsize//3,targetsize*2//3):
    smokinglist.append(2)    
for i in range(targetsize*2//3,targetsize):
    smokinglist.append(0)  
    
random.shuffle(sexlist)     
random.shuffle(HTNlist)    
random.shuffle(DMlist)
random.shuffle(Dyslipidlist)
random.shuffle(Afiblist)
random.shuffle(CKDlist)
random.shuffle(CVDfirstrelativelist)
random.shuffle(smokinglist)    
df_synthetic['sex']=sexlist
df_synthetic['HTN']=HTNlist
df_synthetic['DM']=DMlist
df_synthetic['Dyslipidemia']=Dyslipidlist
df_synthetic['Afib']=Afiblist
df_synthetic['CKD']=CKDlist
df_synthetic['CVDfamily']=CVDfirstrelativelist
df_synthetic['smoking']=smokinglist

rand_df=df_synthetic

txtlist=[]
for i in range(len(rand_df)):
    txt = ''
    txt += str(int(round(rand_df['age'][i],0)))
    txt += ' year old '
    
    if rand_df['sex'][i]==1:
        txt += 'male, '
    else:
        txt += 'female, '
        
    if rand_df['DM'][i]==1:
        txt += 'with diabetes, '
    else:
        txt += 'without diabetes, '
        
    if rand_df['HTN'][i]==1:
        txt += 'with hypertension, '
    else:
        txt += 'without hypertension, ' 
        
    if rand_df['Dyslipidemia'][i]==1:
        txt += 'with dyslipidemia, '
    else:
        txt += 'without dyslipidemia, ' 
    
    if rand_df['CKD'][i]==1:
        txt += 'with chronic kidney disease, '
    else:
        txt += 'without chronic kidney disease, ' 
        
    if rand_df['Afib'][i]==1:
        txt += 'with history of atrial fibrillation, '
    else:
        txt += 'without history of atrial fibrillation, ' 
        
    if rand_df['CVDfamily'][i]==1:
        txt += 'with family history of cardiovascular disease in first degree relatives, '
    else:
        txt += 'without family history of cardiovascular disease in first degree relatives, ' 
        
    if rand_df['smoking'][i]==2:
        txt += 'current smoker, '
    elif rand_df['smoking'][i]==1:
        txt += 'ex-smoker, '
    else:
        txt += 'non smoker, ' 
        
    #txt += 'total cholesterol '
    #txt += str(round(rand_df['Chol'][i],1))
    #txt += ' mg/dL, '
    
    txt += 'HDL '
    txt += str(round(rand_df['HDL'][i],1))
    txt += ' mg/dL, '
    
    txt += 'LDL '
    txt += str(round(rand_df['LDL'][i],1))
    txt += ' mg/dL, '
    
    txt += 'triglyceride '
    txt += str(round(rand_df['TG'][i],1))
    txt += ' mg/dL, '
    
    txt += 'HbA1c '
    txt += str(round(rand_df['HbA1c'][i],2))
    txt += ' %, '
    
    txt += 'creatinine '
    txt += str(round(rand_df['Cr'][i],2))
    txt += ' mg/dL, '
    
    txt += 'urate '
    txt += str(round(rand_df['Urate'][i],2))
    txt += ' mg/dL, '
    
    txt += 'C-reactive protein '
    txt += str(round(rand_df['CRP'][i],2))
    txt += ' mg/dL, '
    
    txt += 'systolic blood pressure '
    txt += str(int(round(rand_df['SBP'][i],0)))
    txt += ' mmHg, '
    
    txt += 'diastolic blood pressure '
    txt += str(int(round(rand_df['DBP'][i],0)))
    txt += ' mmHg, '
    
    txt += 'BMI '
    txt += str(round(rand_df['BMI'][i],2))
    txt += ', '
    
    txt += 'waist to hip ratio '
    txt += str(round(rand_df['WHratio'][i],2))
    txtlist.append(txt)
    
rand_df['input_text']=txtlist

rand_df['age']= round(rand_df['age'], 0)
rand_df['HDL']= round(rand_df['HDL'], 1)
rand_df['LDL']= round(rand_df['LDL'], 1)
rand_df['TG']= round(rand_df['TG'], 1)
rand_df['HbA1c']= round(rand_df['HbA1c'], 2)
rand_df['Cr']= round(rand_df['Cr'], 2)
rand_df['Urate']= round(rand_df['Urate'], 2)
rand_df['CRP']= round(rand_df['CRP'], 2)
rand_df['SBP']= round(rand_df['SBP'], 0)
rand_df['DBP']= round(rand_df['DBP'], 0)
rand_df['BMI']= round(rand_df['BMI'], 2)
rand_df['WHratio']= round(rand_df['WHratio'], 2)

rand_df['eid']=list(rand_df.index)

In [ ]:
rand_df.to_csv('20231226_synthetic_20000.csv',index=False)

In [ ]:
txtlist[1]

# GPT

In [ ]:
rand_df=pd.read_csv('20231226_synthetic_20000.csv')

In [ ]:
usercontent_global1='''Estimate the risk (in percentages) of developing a cardiovascular disease within 10 years for the person below.

'''
usercontent_global2='''

Please answer exactly in the format below, without blank lines, and no further information or answer is required.
Risk percentage=(in percentages, round to one decimal place)'''

input_text=rand_df['input_text']
eid=rand_df['eid']

def timeout(timeout):
    def deco(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            res = [Exception('function [%s] timeout [%s seconds] exceeded!' % (func.__name__, timeout))]
            def newFunc():
                try:
                    res[0] = func(*args, **kwargs)
                except Exception as e:
                    res[0] = e
            t = Thread(target=newFunc)
            t.daemon = True
            try:
                t.start()
                t.join(timeout)
            except Exception as je:
                print ('error starting thread')
                raise je
            ret = res[0]
            #if isinstance(ret, BaseException):
            #    raise ret
            return ret
        return wrapper
    return deco

@timeout(0.1)
def ChatGPT_main(i,temper,responsenum):
    if input_text[i]!='':
        try:
            usercontent=usercontent_global1
            usercontent+=input_text[i]
            usercontent+=usercontent_global2

            messages=[
                    #{"role": "system", "content": usercontent_global3},
                    {"role": "user", "content": usercontent}
                ]
            
            client = OpenAI()
            completion=client.chat.completions.create(
                model='gpt-4',
                n=responsenum,
                temperature=temper,
                messages=messages
            )

            aa2=completion.choices
            for j in range(len(aa2)):
                aa=aa2[j].message.content.split('\n')
            #print(aa)
                remove_set={''}
                aaa=[ii for ii in aa if ii not in remove_set]

                temp1='blank'
                temp2=aaa[0].split('=')[1]
                temp3='blank'
                temp4='blank'
                temp5='blank'
                
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                with conn:
                    with conn.cursor() as cur:
                        cur.execute(sql, (str(eid[i]),temp1,temp2,temp3,temp4,temp5))
                        conn.commit()

        except:
            temp1=np.nan
            temp2=np.nan
            temp3=np.nan
            temp4=np.nan
            temp5=np.nan
            
    else:
        temp1=np.nan
        temp2=np.nan
        temp3=np.nan
        temp4=np.nan
        temp5=np.nan
        


In [ ]:
input_text[4]

In [ ]:
i=4
temper=1.4
time1=datetime.datetime.now()
usercontent=usercontent_global1
usercontent+=input_text[i]
usercontent+=usercontent_global2

messages=[
        #{"role": "system", "content": usercontent_global3},
        {"role": "user", "content": usercontent}
    ]

completion=openai.ChatCompletion.create(model="gpt-4", 
                                        messages=messages,
                                        temperature=temper,
                                        n=5)

aa2=completion.choices
time2=datetime.datetime.now()
print(time2-time1)
#data_final_groupmean[data_final_groupmean['eid']==i]

In [ ]:
completion

In [ ]:
############################
table_name="20231226_gpt4_temperature"
iterations=1
upto=20000
############################
for temper in [0.0,0.2,0.4,0.6,0.8,1.0]:
    time1=datetime.datetime.now()
    while True:
        toggle=0
        print('=========================================================')
        print('temperature',temper)
        for i in range(iterations):   
            newtablaname=table_name + '_' + str(int(temper*10)) + '_' + str(i)
            try:
                sql_createtable="CREATE TABLE [" + newtablaname +"""] 
                (
                    eid    NVARCHAR(20),
                    system    NVARCHAR(max),
                    score      NVARCHAR(max),
                    category     NVARCHAR(max) ,
                    framingham  NVARCHAR(20),
                    ACC_AHA  NVARCHAR(20)
                )

                """
                conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
                with conn:
                    with conn.cursor() as cur:
                        cur.execute(sql_createtable)
                        conn.commit()
                time.sleep(1)  
            except:
                pass
            
            sql_statement="select * from ["+ newtablaname + "]"
            conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
            datasql = pd.read_sql(sql=sql_statement, con=conn)
            datasql=datasql.astype({'eid':int})
            cnt_df=datasql.groupby('eid')['score'].count()
            tempcount=0
            for j in range(upto):
                try:
                    cnt=cnt_df[j]
                    if cnt<10:
                        addcnt=int(10-cnt)
                        sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                        ChatGPT_main(j,temper,addcnt)
                        tempcount+=1

                except:
                    sql = "INSERT INTO [" + newtablaname+"] (eid, system,score,category,framingham,ACC_AHA) VALUES (%s, %s, %s, %s, %s, %s)"
                    ChatGPT_main(j,temper,10)
                    tempcount+=1
                if j%100==0:
                    print(j)
            
            if tempcount==0:
                toggle=1
        if toggle==1:
            break
        time.sleep(20)

    time.sleep(20)


    time2=datetime.datetime.now()
    print(time2-time1)
    #print('except_else_num = ',except_else_num)

# UKB data

In [ ]:
############################
table_name="20231226_gpt4_temperature"
iterations=1
upto=20000

############################
resultsdf=pd.DataFrame(columns=['validnum','age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Cr', 'Urate', 'CRP', 'SBP', 'DBP', 'BMI', 'WHratio', 'sex', 'HTN', 'DM', 'Dyslipidemia', 'Afib', 'CKD', 'CVDfamily', 'smoking','avg_gpt_score','c-index gpt','c-index acc/aha','c-index framingham'])


for temper in [0.0,0.2,0.4,0.6,0.8,1.0]:
    templist=[]
    print('=========temperature========',temper)
    for i in range(iterations):
        newtablename=table_name + '_' + str(int(temper*10)) + '_' + str(i)

        sql_statement="select * from ["+ newtablename + "]"
        conn = pymssql.connect(host=, database=, charset=) # enter pymssql connection info
        globals()['data{}'.format(i)] = pd.read_sql(sql=sql_statement, con=conn)


    for j in range(iterations):
        dat=globals()['data{}'.format(j)]

        eid=dat['eid']
        score_gpt=dat['score']
        category=dat['category']
        score_framingham=dat['framingham']
        score_acc_aha=dat['ACC_AHA']

        score_gpt2=[]

        for i in range(len(score_gpt)):
            try:
                score_gpt2.append(float(re.findall("\d+[.]\d+[%]",score_gpt[i])[0].split('%')[0]))
            except:
                try:
                    score_gpt2.append(float(re.findall("\d+[%]",score_gpt[i])[0].split('%')[0]))
                except:
                    try:
                        score_gpt2.append(float(re.findall("\d+[.]\d+",score_gpt[i])[0]))
                    except:
                        try:
                            score_gpt2.append(float(re.findall("\d+",score_gpt[i])[-1]))
                        except:
                            score_gpt2.append(np.nan)
                            #score_gpt2.append(float(score_framingham[i]))
                            #print(score_gpt[i],'====')


    data_final=pd.DataFrame({'eid':data0['eid'],'score':score_gpt2})
    data_final['eid']=data_final['eid'].astype(int)
    print(len(data_final))
    data_final=data_final.dropna()
    data_final.reset_index(inplace=True,drop=True)

    rand_df=pd.read_csv('20231226_synthetic_20000.csv')
    rand_df=rand_df.iloc[:upto]

    #data_final_groupmean=data_final.groupby(['eid'],as_index=False).mean()

    df_merged=pd.merge(data_final,rand_df,how='inner',on='eid')
    print(len(df_merged))
    templist.append(len(df_merged))
    columnslisttemp=list(df_merged.columns)
    columnslisttemp=columnslisttemp[2:]
    columnslisttemp=columnslisttemp[:-1]
    print(columnslisttemp)
    #X=df_merged[['age','sex','HDL','LDL','TG','Chol','SBP','DBP','BMI','HTN','DM','smoking']]
    X=df_merged[columnslisttemp]
    y=df_merged['score']

    line_fitter = LinearRegression()
    line_fitter.fit(X, y)

    print(line_fitter.coef_)
    templist+=list(line_fitter.coef_)

    df_final=pd.read_csv('processed_UKB_data.csv') # the processed UKB data should be imported

    columnslist=[]
    columnslist.append('age')
    columnslist.append('sex')
    #columnslist.append('total_chol')
    columnslist.append('HDL')
    columnslist.append('LDL')
    columnslist.append('TG')
    columnslist.append('SBP')
    columnslist.append('DBP')
    columnslist.append('med_final')
    columnslist.append('DM_final')
    columnslist.append('BMI_x')
    columnslist.append('cig_final')
    columnslist.append('E78_dyslipidemia_binary')
    columnslist.append('I48_afib_binary')
    columnslist.append('N18_CKD_binary')
    columnslist.append('HbA1c')
    columnslist.append('Creatinine')
    columnslist.append('urate')
    columnslist.append('C_reactive_ptn')
    columnslist.append('CVD_firstdegree')
    columnslist.append('firstoccur_event')
    columnslist.append('WHratio')
    columnslist.append('eid')
    columnslist.append('Framingham')
    columnslist.append('ACC/AHA')
    columnslist.append('datediff_firstoccur_int2')
    df_final2=df_final[columnslist]
    df_final2.dropna(inplace=True)
    df_final2.reset_index(inplace=True,drop=True)
    firstoccur_event=[]
    for i in range(len(df_final2)):
        if df_final2['firstoccur_event'][i]==True:
            firstoccur_event.append(1)
        else:
            firstoccur_event.append(0)
    df_final2['firstoccur_event']=firstoccur_event

    df2=df_final2[['eid','firstoccur_event','datediff_firstoccur_int2','ACC/AHA','Framingham','age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Creatinine', 'urate', 'C_reactive_ptn', 'SBP', 'DBP', 'BMI_x', 'WHratio', 'sex', 'med_final', 'DM_final', 'E78_dyslipidemia_binary', 'I48_afib_binary', 'N18_CKD_binary', 'CVD_firstdegree', 'cig_final']]
    df2=df2.dropna()
    print(len(df2))

    X=df2[['age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Creatinine', 'urate','C_reactive_ptn', 'SBP', 'DBP', 'BMI_x', 'WHratio', 'sex', 'med_final', 'DM_final', 'E78_dyslipidemia_binary', 'I48_afib_binary', 'N18_CKD_binary', 'CVD_firstdegree', 'cig_final']]
    X.columns=['age', 'HDL', 'LDL', 'TG', 'HbA1c', 'Cr', 'Urate', 'CRP', 'SBP', 'DBP', 'BMI', 'WHratio', 'sex', 'HTN', 'DM', 'Dyslipidemia', 'Afib', 'CKD', 'CVDfamily', 'smoking']
    y=line_fitter.predict(X)
    df2['score']=y
    df_final=df2
    
    templist.append(np.mean(df_final['score']))

    fpr,tpr,thres=roc_curve(df_final['firstoccur_event'],df_final['score'])
    print('event percentage',np.mean(df_final['firstoccur_event']))
    print('average gpt score',np.mean(df_final['score']))
    print('average accaha',np.mean(df_final['ACC/AHA']))
    print('average framingham',np.mean(df_final['Framingham']))
    print('gpt auroc',auc(fpr, tpr))
    fpr,tpr,thres=roc_curve(df_final['firstoccur_event'],df_final['ACC/AHA'])
    print('accaha auroc',auc(fpr, tpr))
    fpr,tpr,thres=roc_curve(df_final['firstoccur_event'],df_final['Framingham'])
    print('framingham auroc',auc(fpr, tpr))
    
    templist.append(concordance_index(df_final['datediff_firstoccur_int2'], -df_final['score'], df_final['firstoccur_event']))
    templist.append(concordance_index(df_final['datediff_firstoccur_int2'], -df_final['ACC/AHA'], df_final['firstoccur_event']))
    templist.append(concordance_index(df_final['datediff_firstoccur_int2'], -df_final['Framingham'], df_final['firstoccur_event']))
    
    resultsdf.loc[len(resultsdf)]=templist
    
    print('c-index gpt',concordance_index(df_final['datediff_firstoccur_int2'], -df_final['score'], df_final['firstoccur_event']))
    print('c-index acc/aha',concordance_index(df_final['datediff_firstoccur_int2'], -df_final['ACC/AHA'], df_final['firstoccur_event']))
    print('c-index framingham',concordance_index(df_final['datediff_firstoccur_int2'], -df_final['Framingham'], df_final['firstoccur_event']))

In [ ]:
resultsdf.index=[0.0,0.2,0.4,0.6,0.8,1.0]

In [ ]:
resultsdf

In [ ]:
fig, axs = plt.subplots(5, 5, figsize=(28, 20))

# Assigning each subplot to a DataFrame column
for i, ax in enumerate(axs.flatten()):
    column_name = resultsdf.columns[i]
    ax.plot(resultsdf.index, resultsdf[column_name])
    ax.set_title(column_name)
    ax.set_xlabel('Index')
    ax.set_ylabel('Values')

plt.tight_layout()
plt.savefig('20231227_temptest.png')
plt.show()